In [51]:
import numpy as np
import pandas as pd
from datetime import datetime

In [20]:
data_folder = '../data/julie_bch_obs/'
df_set_1 = pd.read_csv(f"{data_folder}2023-01-01_2023-12-31_hourly[UTC]_set1.csv")
df_set_2 = pd.read_csv(f"{data_folder}2023-01-01_2023-12-31_hourly[UTC]_set2.csv")
df_set_3 = pd.read_csv(f"{data_folder}2023-01-01_2023-12-31_hourly[UTC]_set3.csv")

In [34]:
print(df_set_1['valid[UTC]'].values[0]) # [11:13]
print(len(df_set_1))

2023-01-01 00:00:00
8760


In [8]:
stations = pd.read_csv(f"{data_folder}station_metadata.csv")
stations.head(10)

,station_no,station_name,site_no,station_longitude,station_latitude,station_elevation
0,85A,85th Avenue,07F,NaN,NaN,NaN
1,YXX,Abbotsford A,08M,-122.360000,49.025278,59.0
2,1F01A,Aberdeen Lk SC,08L,-119.053293,50.146514,1262.0
3,1E07,Adams R SC,08L,-119.426400,51.583611,1720.0
4,1100120,Agassiz CDA,08M,-121.759722,49.242500,15.0
5,WZA,Agassiz CS,08M,-121.750000,49.233333,19.0
6,AKN,Aiken Lk,07E,-125.741667,56.436667,970.0
7,05AD802,Akamina Pass,05A,NaN,NaN,NaN
8,AKM,Akamina Pass 2,05A,-114.053497,49.027307,1809.0
9,AKI,Akie R nr 760 M Contour,07E,-124.904244,57.191192,760.0


In [11]:
stations[stations.station_no=='FAQ']

,station_no,station_name,site_no,station_longitude,station_latitude,station_elevation
313,FAQ,Fauquier Water Treatment Plant,08N,-118.074228,49.864182,519.0


In [21]:
stations_set_1 = [x.split('_')[0] for x in df_set_1.columns if len(x.split('_'))==2]
stations_set_2 = [x.split('_')[0] for x in df_set_2.columns if len(x.split('_'))==2]
stations_set_3 = [x.split('_')[0] for x in df_set_3.columns if len(x.split('_'))==2]
stations_list = stations_set_1 + stations_set_2 + stations_set_3
print(stations_list)

['ALU', 'ALU', 'ASH', 'ASH', 'BAR', 'BAR', 'BCK', 'BCK', 'BGP', 'BGP', 'CDW', 'CDW', 'CLO', 'CLO', 'CMX', 'CMX', 'DAI', 'DAI', 'DLU', 'DLU', 'EAC', 'EAC', 'ECL', 'ECL', 'FAQ', 'FAQ', 'FIN', 'FIN', 'FLL', 'FLL', 'GOC', 'GOC', 'GOL2', 'GOL2', 'HFF', 'HFF', 'HRN', 'HRN', 'HUR2', 'HUR2', 'ING', 'ING', 'JHL', 'JHL', 'KAA', 'KAA', 'KWA', 'KWA', 'MCQ', 'MCQ', 'MIS', 'MIS', 'MLP', 'MLP', 'MOL', 'MOL', 'MOR', 'MOR', 'NDE', 'NDE', 'NTY', 'NTY', 'PAR', 'PAR', 'PKM', 'PKM', 'PYN', 'PYN', 'QIN2', 'QIN2', 'REC', 'REC', 'RST', 'RST', 'SBD2', 'SBD2', 'SHH', 'SHH', 'STV', 'STV', 'UCE', 'UCE', 'WAH', 'WAH', 'WJR', 'WJR', 'WOL', 'WOL', 'WON', 'WON', 'WZA', 'WZA', 'YGS', 'YGS', 'YLW', 'YLW', 'YRV', 'YRV', 'YXS', 'YXS', 'YYD', 'YYD', 'YZY', 'YZY']


In [ ]:
df = pd.DataFrame(columns=['UTC_DATE', 'x', 'y', 'STN_ID', 'TEMP', 'PRECIP_AMOUNT'])

In [55]:
init_hours = [00, 6, 12, 18]
count = 0
for df_data in [df_set_1, df_set_2, df_set_3]:
    for station in [x.split('_')[0] for x in df_data.columns if len(x.split('_'))==2]:
        print(count)
        precip_amount = df_data[f"{station}_Precip[mm]"].rolling(6).sum()

        concat = pd.DataFrame({
            'UTC_DATE': df_data['valid[UTC]'],
            'x': stations[stations.station_no==station]['station_longitude'].values[0],
            'y': stations[stations.station_no==station]['station_latitude'].values[0],
            'STN_ID': station,
            'TEMP': df_data[f"{station}_AirTemp[degC]"],
            'PRECIP_AMOUNT': precip_amount,
        })
        dt = concat.UTC_DATE.apply(lambda x :  datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
        init_mask = dt.apply(lambda x: x.hour in init_hours)

        concat = concat[init_mask]
        df = pd.concat((concat, df), axis=0)
        count+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103


In [56]:
df.to_csv(f"{data_folder}/processed_bch_obs.csv")
df

,UTC_DATE,x,y,STN_ID,TEMP,PRECIP_AMOUNT
0,2023-01-01 00:00:00,-123.138056,55.305278,YZY,-4.9,NaN
6,2023-01-01 06:00:00,-123.138056,55.305278,YZY,-5.1,0.0
12,2023-01-01 12:00:00,-123.138056,55.305278,YZY,-2.9,0.0
18,2023-01-01 18:00:00,-123.138056,55.305278,YZY,-2.0,0.0
24,2023-01-02 00:00:00,-123.138056,55.305278,YZY,-1.5,0.0
...,...,...,...,...,...,...
8730,2023-12-30 18:00:00,-122.486256,49.287453,ALU,8.0,1.0
8736,2023-12-31 00:00:00,-122.486256,49.287453,ALU,8.4,1.4
8742,2023-12-31 06:00:00,-122.486256,49.287453,ALU,7.2,6.8
8748,2023-12-31 12:00:00,-122.486256,49.287453,ALU,6.8,1.0
